In [25]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import re
import requests
import time

pd.set_option('display.max_columns', None)

In [ ]:
ump_scores_df = pd.read_csv('D:\Downloads\games (1).csv')

##drop nulls, shortened season
ump_scores_df = ump_scores_df.dropna(axis='index').sort_values('Date')
ump_scores_df = ump_scores_df.set_axis(['date', 'umpire', 'home', 'away', 'home_score', 'away_score', 'called_pitches', 'called_wrong', 'accuracy', 'consistency', 'favor', 'total_run_impact'], axis=1)
covid_2020 = ump_scores_df.date.str.contains('2020')
ump_scores_df = ump_scores_df[~covid_2020].reset_index(drop=True)

##clean for scraping
ump_scores_df.date = ump_scores_df.date.str.replace('-', '')
ump_scores_df.loc[ump_scores_df.home == 'WSH', 'home'] = 'WAS'
ump_scores_df.loc[ump_scores_df.home == 'LAD', 'home'] = 'LAN'
ump_scores_df.loc[ump_scores_df.home == 'SD', 'home'] = 'SDN'
ump_scores_df.loc[ump_scores_df.home == 'SF', 'home'] = 'SFN'
ump_scores_df.loc[ump_scores_df.home == 'NYY', 'home'] = 'NYA'
ump_scores_df.loc[ump_scores_df.home == 'KC', 'home'] = 'KCA'
ump_scores_df.loc[ump_scores_df.home == 'STL', 'home'] = 'SLN'
ump_scores_df.loc[ump_scores_df.home == 'NYM', 'home'] = 'NYN'
ump_scores_df.loc[ump_scores_df.home == 'TB', 'home'] = 'TBA'
ump_scores_df.loc[ump_scores_df.home == 'CWS', 'home'] = 'CHA'
ump_scores_df.loc[ump_scores_df.home == 'CHC', 'home'] = 'CHN'
display(ump_scores_df)

ump_scores_df.groupby('date')['home_score']

,date,umpire,home,away,home_score,away_score,called_pitches,called_wrong,accuracy,consistency,favor,total_run_impact
0,20150405,Mike Winters,CHN,STL,0,3,158.0,17.0,89.2,94.9,1.10,4.36
1,20150406,John Hirschbeck,CIN,PIT,5,2,117.0,18.0,84.6,87.2,-0.45,2.21
2,20150406,Joe West,DET,MIN,4,0,109.0,8.0,92.7,96.3,-0.50,0.84
3,20150406,Gerry Davis,PHI,BOS,0,8,171.0,18.0,89.5,94.7,-1.00,2.84
4,20150406,Ted Barrett,HOU,CLE,2,0,124.0,21.0,83.1,91.9,0.51,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...
13484,20211027,Ron Kulpa,HOU,ATL,7,2,120.0,5.0,95.8,96.7,0.10,0.50
13485,20211029,Tom Hallion,ATL,HOU,2,0,146.0,5.0,96.6,93.8,-0.09,0.41
13486,20211030,Dan Bellino,ATL,HOU,3,2,153.0,14.0,90.8,94.1,-0.26,1.48
13487,20211031,Ted Barrett,ATL,HOU,5,9,171.0,14.0,91.8,88.3,0.61,1.47


In [52]:
##webscrape to pull start_time, attendance, duration, day_night
url = 'https://www.baseball-reference.com/boxes/'

scorebox = []
start_time = []
duration = []
day_night = []
for i, j in zip(ump_scores_df.date, ump_scores_df.home):
    response = requests.get(url + j + '/' + j + i + '0.shtml')
    soup = bs(response.text, 'html.parser')
    print(j)
    print(i)
    print(response)
    scorebox = soup.find_all('div', {'class':'scorebox_meta'})
    start_time.append(scorebox)
    duration.append(scorebox)
    day_night.append(scorebox)
    time.sleep(3)

##estimated time to run: ~11 hours
##started: 10:34 AM EDT
##ended: 11:12 PM EDT

CHN
20150405
<Response [200]>
CIN
20150406
<Response [200]>
DET
20150406
<Response [200]>
PHI
20150406
<Response [200]>
HOU
20150406
<Response [200]>
ARI
20150406
<Response [200]>
OAK
20150406
<Response [200]>
LAN
20150406
<Response [200]>
TBA
20150406
<Response [200]>
KCA
20150406
<Response [200]>
MIL
20150406
<Response [200]>
NYA
20150406
<Response [200]>
MIA
20150406
<Response [200]>
WAS
20150406
<Response [200]>
SEA
20150406
<Response [200]>
TBA
20150407
<Response [200]>
MIL
20150407
<Response [200]>
OAK
20150407
<Response [200]>
MIA
20150407
<Response [200]>
SEA
20150407
<Response [200]>
ARI
20150407
<Response [200]>
LAN
20150408
<Response [200]>
HOU
20150408
<Response [200]>
SEA
20150408
<Response [200]>
PHI
20150408
<Response [200]>
MIL
20150408
<Response [200]>
NYA
20150408
<Response [200]>
DET
20150408
<Response [200]>
KCA
20150408
<Response [200]>
OAK
20150408
<Response [200]>
TBA
20150408
<Response [200]>
CIN
20150408
<Response [200]>
WAS
20150408
<Response [200]>
CHN
201504

In [57]:
##save lists to dataframe then export as csv to be used elsewhere
start_time_df = pd.DataFrame(start_time)
duration_df = pd.DataFrame(duration)
day_night_df = pd.DataFrame(day_night)
start_time_df.to_csv('start_time_df.csv')
duration_df.to_csv('duration_df.csv')
day_night_df.to_csv('day_night_df.csv')